Block 1: Setup and Imports

In [33]:
# Cell 1: Setup and Imports
import os
import json
import pickle
import warnings
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any
from pathlib import Path

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Core ML libraries
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
import openai

# Jupyter widgets
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from IPython.display import Audio, Image

# Environment and utilities
from dotenv import load_dotenv
import requests
from tqdm.notebook import tqdm
import faiss

# Load environment variables
load_dotenv('config.env')

print("✅ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

✅ All libraries imported successfully!
PyTorch version: 2.7.1
CUDA available: False


Block 2: Configuration and Constants

In [34]:
# Cell 2: Configuration and Constants
class Config:
    """Configuration class for the multilingual learning assistant"""
    
    # API Keys
    HUGGINGFACE_TOKEN = os.getenv('HUGGINGFACE_API_TOKEN')
    GOOGLE_AI_KEY = os.getenv('GOOGLE_AI_API_KEY')
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    
    # File paths
    FAISS_INDEX_PATH = os.getenv('FAISS_INDEX_PATH', './data/word_embeddings.faiss')
    METADATA_PATH = os.getenv('METADATA_PATH', './data/learned_words.json')
    
    # Model configurations (Updated)
    DEFAULT_TRANSLATION_MODEL = "Helsinki-NLP/opus-mt-hi-en"  # Smaller, reliable model
    
    # Language mappings
    LANGUAGE_MAPPINGS = {
        'hindi': {'code': 'hi', 'name': 'Hindi', 'script': 'Devanagari'},
        'tamil': {'code': 'ta', 'name': 'Tamil', 'script': 'Tamil'},
        'bengali': {'code': 'bn', 'name': 'Bengali', 'script': 'Bengali'},
        'kannada': {'code': 'kn', 'name': 'Kannada', 'script': 'Kannada'},
        'telugu': {'code': 'te', 'name': 'Telugu', 'script': 'Telugu'},
        'gujarati': {'code': 'gu', 'name': 'Gujarati', 'script': 'Gujarati'},
        'marathi': {'code': 'mr', 'name': 'Marathi', 'script': 'Devanagari'},
        'punjabi': {'code': 'pa', 'name': 'Punjabi', 'script': 'Gurmukhi'},
        'urdu': {'code': 'ur', 'name': 'Urdu', 'script': 'Perso-Arabic'},
        'malayalam': {'code': 'ml', 'name': 'Malayalam', 'script': 'Malayalam'},
        'odia': {'code': 'or', 'name': 'Odia', 'script': 'Odia'},
        'assamese': {'code': 'as', 'name': 'Assamese', 'script': 'Bengali'},
        'manipuri': {'code': 'mni', 'name': 'Manipuri', 'script': 'Bengali'},
        'kashmiri': {'code': 'ks', 'name': 'Kashmiri', 'script': 'Perso-Arabic'},
        'konkani': {'code': 'kok', 'name': 'Konkani', 'script': 'Devanagari'},
        'sanskrit': {'code': 'sa', 'name': 'Sanskrit', 'script': 'Devanagari'}
    }
    
    # Create data directory
    @classmethod
    def setup_directories(cls):
        Path('./data').mkdir(exist_ok=True)
        Path('./logs').mkdir(exist_ok=True)

# Initialize configuration
Config.setup_directories()
print("✅ Configuration loaded successfully!")
print(f"Supported languages: {len(Config.LANGUAGE_MAPPINGS)}")

✅ Configuration loaded successfully!
Supported languages: 16


Block 3: Translation Service

In [35]:
# Cell 3: Translation Service
class TranslationService:
    """Handles translation between Indian languages and English"""
    
    def __init__(self):
        self.translator = None
        self._initialize_translators()
    
    def _initialize_translators(self):
        """Initialize translation models with error handling"""
        try:
            print("🔄 Initializing translation models...")
            
            # Use a smaller, more reliable model
            model_name = "Helsinki-NLP/opus-mt-hi-en"  # Hindi to English (smaller, reliable)
            
            if Config.HUGGINGFACE_TOKEN:
                self.translator = pipeline(
                    "translation",
                    model=model_name,
                    token=Config.HUGGINGFACE_TOKEN
                )
                print(f"✅ Translator loaded: {model_name}")
            else:
                print("⚠️ No Hugging Face token provided, trying without token")
                self.translator = pipeline(
                    "translation",
                    model=model_name
                )
                print(f"✅ Translator loaded: {model_name}")
                
        except Exception as e:
            print(f"❌ Error loading translator: {e}")
            self.translator = None
    
    def translate_to_english(self, text: str, source_language: str) -> Dict[str, Any]:
        """Translate text to English with metadata"""
        result = {
            'original_text': text,
            'source_language': source_language,
            'translated_text': 'unknown',
            'confidence': 0.0,
            'model_used': 'none',
            'error': None
        }
        
        try:
            if self.translator:
                translation = self.translator(text)
                result['translated_text'] = translation[0]['translation_text']
                result['confidence'] = 0.7  # Lower confidence since it's not language-specific
                result['model_used'] = 'Helsinki-NLP/opus-mt-hi-en'
            else:
                result['error'] = "No translation models available"
                
        except Exception as e:
            result['error'] = str(e)
            print(f"❌ Translation error: {e}")
        
        return result

# Initialize translation service
translation_service = TranslationService()

🔄 Initializing translation models...


Device set to use mps:0


✅ Translator loaded: Helsinki-NLP/opus-mt-hi-en


Block 4: Gemma 3n Integration


In [36]:
# Cell 4: Gemma 3n Integration (Fixed)
class GemmaService:
    """Handles interactions with Gemma 3n model"""
    
    def __init__(self):
        self.model = None
        self._initialize_gemma()
    
    def _initialize_gemma(self):
        """Initialize Gemma 3n model"""
        try:
            if Config.GOOGLE_AI_KEY:
                genai.configure(api_key=Config.GOOGLE_AI_KEY)
                # Use the correct model name from the available list
                self.model = genai.GenerativeModel('gemma-3n-e4b-it')  # Using the 4B model
                print("Gemma 3n model initialized successfully!")
            else:
                print("No Google AI API key provided")
                self.model = None
        except Exception as e:
            print(f"Error initializing Gemma: {e}")
            self.model = None
    
    def get_word_explanation(self, word: str, language: str, english_translation: str) -> Dict[str, Any]:
        """Get comprehensive explanation from Gemma"""
        if not self.model:
            return {
                'explanation': 'Gemma model not available',
                'pronunciation': 'unknown',
                'usage_examples': [],
                'cultural_context': 'unknown',
                'error': 'Model not initialized'
            }
        
        try:
            prompt = f"""
            Please provide a comprehensive explanation for the word "{word}" from {language} language.
            
            English translation: {english_translation}
            
            Please provide:
            1. Pronunciation guide (with IPA if possible)
            2. Usage examples in sentences
            3. Cultural context and significance
            4. Related words or synonyms
            
            Format your response as JSON with these keys:
            - pronunciation
            - usage_examples (list)
            - cultural_context
            - related_words (list)
            """
            
            response = self.model.generate_content(prompt)
            
            # Try to parse JSON response
            try:
                import json
                explanation_data = json.loads(response.text)
                return {
                    'explanation': response.text,
                    'pronunciation': explanation_data.get('pronunciation', 'unknown'),
                    'usage_examples': explanation_data.get('usage_examples', []),
                    'cultural_context': explanation_data.get('cultural_context', 'unknown'),
                    'related_words': explanation_data.get('related_words', []),
                    'error': None
                }
            except:
                # If JSON parsing fails, return raw response
                return {
                    'explanation': response.text,
                    'pronunciation': 'unknown',
                    'usage_examples': [],
                    'cultural_context': 'unknown',
                    'related_words': [],
                    'error': None
                }
                
        except Exception as e:
            return {
                'explanation': 'Error generating explanation',
                'pronunciation': 'unknown',
                'usage_examples': [],
                'cultural_context': 'unknown',
                'related_words': [],
                'error': str(e)
            }

# Initialize Gemma service
gemma_service = GemmaService()

Gemma 3n model initialized successfully!


Block 5: FAISS Storage Service

In [37]:
# Cell 5: FAISS Storage Service
class WordStorageService:
    """Handles storage and retrieval of learned words using FAISS"""
    
    def __init__(self):
        self.index = None
        self.metadata = []
        self.embedding_model = None
        self._initialize_storage()
    
    def _initialize_storage(self):
        """Initialize FAISS index and embedding model"""
        try:
            # Load embedding model
            self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
            print("✅ Embedding model loaded")
            
            # Try to load existing index
            if os.path.exists(Config.FAISS_INDEX_PATH):
                self.index = faiss.read_index(Config.FAISS_INDEX_PATH)
                print(f"✅ Loaded existing FAISS index with {self.index.ntotal} vectors")
            else:
                # Create new index
                dimension = self.embedding_model.get_sentence_embedding_dimension()
                self.index = faiss.IndexFlatL2(dimension)
                print(f"✅ Created new FAISS index with dimension {dimension}")
            
            # Load metadata
            self._load_metadata()
            
        except Exception as e:
            print(f"❌ Error initializing storage: {e}")
    
    def _load_metadata(self):
        """Load word metadata from JSON file"""
        try:
            if os.path.exists(Config.METADATA_PATH):
                with open(Config.METADATA_PATH, 'r', encoding='utf-8') as f:
                    self.metadata = json.load(f)
                print(f"✅ Loaded {len(self.metadata)} word entries")
            else:
                self.metadata = []
                print("✅ No existing metadata found, starting fresh")
        except Exception as e:
            print(f"❌ Error loading metadata: {e}")
            self.metadata = []
    
    def _save_metadata(self):
        """Save word metadata to JSON file"""
        try:
            with open(Config.METADATA_PATH, 'w', encoding='utf-8') as f:
                json.dump(self.metadata, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"❌ Error saving metadata: {e}")
    
    def add_word(self, word_data: Dict[str, Any]) -> bool:
        """Add a new word to storage"""
        try:
            # Create embedding
            text_for_embedding = f"{word_data['original_word']} {word_data['english_translation']} {word_data['language']}"
            embedding = self.embedding_model.encode([text_for_embedding])
            
            # Add to FAISS index
            self.index.add(embedding)
            
            # Add metadata
            word_entry = {
                'id': len(self.metadata),
                'original_word': word_data['original_word'],
                'english_translation': word_data['english_translation'],
                'language': word_data['language'],
                'pronunciation': word_data.get('pronunciation', 'unknown'),
                'usage_examples': word_data.get('usage_examples', []),
                'cultural_context': word_data.get('cultural_context', 'unknown'),
                'related_words': word_data.get('related_words', []),
                'date_added': datetime.now().isoformat(),
                'translation_confidence': word_data.get('translation_confidence', 0.0),
                'model_used': word_data.get('model_used', 'unknown')
            }
            
            self.metadata.append(word_entry)
            
            # Save to disk
            faiss.write_index(self.index, Config.FAISS_INDEX_PATH)
            self._save_metadata()
            
            print(f"✅ Added word: {word_data['original_word']} ({word_data['language']})")
            return True
            
        except Exception as e:
            print(f"❌ Error adding word: {e}")
            return False
    
    def search_words(self, query: str, k: int = 5) -> List[Dict[str, Any]]:
        """Search for similar words"""
        try:
            query_embedding = self.embedding_model.encode([query])
            distances, indices = self.index.search(query_embedding, min(k, len(self.metadata)))
            
            results = []
            for i, idx in enumerate(indices[0]):
                if idx < len(self.metadata):
                    result = self.metadata[idx].copy()
                    result['similarity_score'] = 1.0 / (1.0 + distances[0][i])
                    results.append(result)
            
            return results
        except Exception as e:
            print(f"❌ Error searching words: {e}")
            return []
    
    def get_all_words(self) -> List[Dict[str, Any]]:
        """Get all stored words"""
        return self.metadata.copy()
    
    def get_words_by_language(self, language: str) -> List[Dict[str, Any]]:
        """Get words by specific language"""
        return [word for word in self.metadata if word['language'].lower() == language.lower()]
    
    def get_random_words_for_quiz(self, count: int = 5, languages: List[str] = None) -> List[Dict[str, Any]]:
        """Get random words for quiz mode"""
        import random
        
        if languages:
            available_words = [word for word in self.metadata if word['language'].lower() in [lang.lower() for lang in languages]]
        else:
            available_words = self.metadata
        
        if len(available_words) < count:
            return available_words
        
        return random.sample(available_words, count)

# Initialize storage service
storage_service = WordStorageService()

✅ Embedding model loaded
✅ Loaded existing FAISS index with 14 vectors
✅ Loaded 14 word entries


Block 6: Main Learning Interface

In [38]:
# Cell 6: Main Learning Interface
class MultilingualLearningInterface:
    """Main interface for the multilingual learning assistant"""
    
    def __init__(self):
        self.setup_widgets()
        self.setup_layout()
    
    def setup_widgets(self):
        """Setup all interactive widgets with enhanced Unicode support"""
        
        # Language selection
        self.language_dropdown = widgets.Dropdown(
            options=[(f"{lang_info['name']} ({lang_info['script']})", lang) 
                    for lang, lang_info in Config.LANGUAGE_MAPPINGS.items()],
            description='Language:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Create a custom HTML input for better Unicode support
        self.word_input_html = widgets.HTML(
            value='''
            <input type="text" id="unicode-input" 
                placeholder="Type a word or phrase..." 
                style="width: 400px; height: 30px; padding: 5px; border: 1px solid #ccc; border-radius: 3px;"
                onchange="window.unicodeInputValue = this.value;">
            ''',
            layout=widgets.Layout(width='420px')
        )
        
        # Create a hidden text widget to store the value
        self.word_input = widgets.Text(
            value='',
            disabled=True,
            layout=widgets.Layout(display='none')
        )
        
        # Add JavaScript to sync the HTML input with the hidden widget
        display(HTML('''
        <script>
        document.getElementById('unicode-input').addEventListener('input', function() {
            window.unicodeInputValue = this.value;
        });
        </script>
        '''))
        
        # Learn button
        self.learn_button = widgets.Button(
            description='Learn Word',
            button_style='primary',
            layout=widgets.Layout(width='150px')
        )
        self.learn_button.on_click(self.learn_word)
        
        # Quiz button
        self.quiz_button = widgets.Button(
            description='Start Quiz',
            button_style='success',
            layout=widgets.Layout(width='150px')
        )
        self.quiz_button.on_click(self.start_quiz)
        
        # Search button
        self.search_button = widgets.Button(
            description='Search Words',
            button_style='info',
            layout=widgets.Layout(width='150px')
        )
        self.search_button.on_click(self.search_words)
        
        # Progress display
        self.progress_output = widgets.Output()
        
        # Results display
        self.results_output = widgets.Output()
        
        # Status display
        self.status_output = widgets.Output()
            
        
    def setup_layout(self):
        """Setup the layout of widgets"""
        
        # Header
        header = widgets.HTML(
            value="<h1>🌍 Multilingual Learning Assistant</h1><p>Learn words from Indian languages with AI-powered explanations</p>",
            layout=widgets.Layout(margin='10px 0px')
        )
        
        # Input section
        input_section = widgets.VBox([
            widgets.HBox([self.language_dropdown, self.word_input]),
            widgets.HBox([self.learn_button, self.quiz_button, self.search_button])
        ])
        
        # Display section
        display_section = widgets.VBox([
            self.status_output,
            self.progress_output,
            self.results_output
        ])
        
        # Main layout
        self.main_layout = widgets.VBox([
            header,
            input_section,
            display_section
        ])
    
    def display(self):
        """Display the interface"""
        display(self.main_layout)
    
    def update_status(self, message: str, status_type: str = 'info'):
        """Update status display"""
        with self.status_output:
            clear_output(wait=True)
            if status_type == 'error':
                display(HTML(f"<p style='color: red;'>{message}</p>"))
            elif status_type == 'success':
                display(HTML(f"<p style='color: green;'>{message}</p>"))
            else:
                display(HTML(f"<p style='color: blue;'>{message}</p>"))
    
    def learn_word(self, button):
        """Process word learning workflow"""
        with self.progress_output:
            clear_output(wait=True)
            display(HTML("<p>🔄 Processing word...</p>"))
        
        try:
            # Get inputs
            language = self.language_dropdown.value
            word = self.word_input.value.strip()
            
            if not language or not word:
                self.update_status("Please select a language and enter a word", 'error')
                return
            
            # Step 1: Translate
            self.update_status("Translating word to English...")
            translation_result = translation_service.translate_to_english(word, language)
            
            if translation_result['error']:
                self.update_status(f"Translation failed: {translation_result['error']}", 'error')
                return
            
            # Step 2: Get Gemma explanation
            self.update_status("Getting AI explanation...")
            explanation_result = gemma_service.get_word_explanation(
                word, language, translation_result['translated_text']
            )
            
            # Step 3: Store word
            self.update_status("Saving word to database...")
            word_data = {
                'original_word': word,
                'english_translation': translation_result['translated_text'],
                'language': language,
                'pronunciation': explanation_result['pronunciation'],
                'usage_examples': explanation_result['usage_examples'],
                'cultural_context': explanation_result['cultural_context'],
                'related_words': explanation_result['related_words'],
                'translation_confidence': translation_result['confidence'],
                'model_used': translation_result['model_used']
            }
            
            success = storage_service.add_word(word_data)
            
            if success:
                # Display results
                with self.results_output:
                    clear_output(wait=True)
                    
                    html_content = f"""
                    <div style='border: 2px solid #4CAF50; padding: 15px; border-radius: 10px; margin: 10px 0;'>
                        <h3>✅ Word Learned Successfully!</h3>
                        <p><strong>Original:</strong> {word} ({Config.LANGUAGE_MAPPINGS[language]['name']})</p>
                        <p><strong>Translation:</strong> {translation_result['translated_text']}</p>
                        <p><strong>Pronunciation:</strong> {explanation_result['pronunciation']}</p>
                        
                        <h4>Usage Examples:</h4>
                        <ul>
                    """
                    
                    for example in explanation_result['usage_examples'][:3]:
                        html_content += f"<li>{example}</li>"
                    
                    html_content += f"""
                        </ul>
                        
                        <h4>Cultural Context:</h4>
                        <p>{explanation_result['cultural_context']}</p>
                        
                        <h4>Related Words:</h4>
                        <ul>
                    """
                    
                    for related in explanation_result['related_words'][:3]:
                        html_content += f"<li>{related}</li>"
                    
                    html_content += """
                        </ul>
                    </div>
                    """
                    
                    display(HTML(html_content))
                
                self.update_status(f"Successfully learned '{word}' from {Config.LANGUAGE_MAPPINGS[language]['name']}!", 'success')
                self.word_input.value = ''  # Clear input
            else:
                self.update_status("Failed to save word", 'error')
                
        except Exception as e:
            self.update_status(f"Error: {str(e)}", 'error')
    
    def start_quiz(self, button):
        """Start quiz mode"""
        quiz_words = storage_service.get_random_words_for_quiz(5)
        
        if not quiz_words:
            self.update_status("No words available for quiz. Learn some words first!", 'error')
            return
        
        self.run_quiz(quiz_words)
    
    def run_quiz(self, quiz_words: List[Dict[str, Any]]):
        """Run the quiz interface"""
        with self.results_output:
            clear_output(wait=True)
            
            html_content = "<h3>🧠 Quiz Mode</h3>"
            
            for i, word in enumerate(quiz_words, 1):
                html_content += f"""
                <div style='border: 1px solid #ddd; padding: 10px; margin: 10px 0; border-radius: 5px;'>
                    <h4>Question {i}:</h4>
                    <p><strong>Word:</strong> {word['original_word']} ({word['language']})</p>
                    <p><strong>What does this word mean in English?</strong></p>
                    <details>
                        <summary>Click to see answer</summary>
                        <p><strong>Answer:</strong> {word['english_translation']}</p>
                        <p><strong>Pronunciation:</strong> {word['pronunciation']}</p>
                        <p><strong>Cultural Context:</strong> {word['cultural_context']}</p>
                    </details>
                </div>
                """
            
            display(HTML(html_content))
        
        self.update_status(f"Quiz completed! Tested {len(quiz_words)} words.", 'success')
    
    def search_words(self, button):
        """Search existing words"""
        search_query = self.word_input.value.strip()
        
        if not search_query:
            self.update_status("Please enter a search term", 'error')
            return
        
        results = storage_service.search_words(search_query, 10)
        
        with self.results_output:
            clear_output(wait=True)
            
            if results:
                html_content = f"<h3>�� Search Results for '{search_query}'</h3>"
                
                for word in results:
                    html_content += f"""
                    <div style='border: 1px solid #ddd; padding: 10px; margin: 10px 0; border-radius: 5px;'>
                        <p><strong>{word['original_word']}</strong> ({word['language']})</p>
                        <p><strong>Translation:</strong> {word['english_translation']}</p>
                        <p><strong>Similarity Score:</strong> {word['similarity_score']:.2f}</p>
                        <p><strong>Added:</strong> {word['date_added'][:10]}</p>
                    </div>
                    """
                
                display(HTML(html_content))
                self.update_status(f"Found {len(results)} similar words", 'success')
            else:
                display(HTML("<p>No words found matching your search.</p>"))
                self.update_status("No results found", 'info')

# Initialize and display the interface
interface = MultilingualLearningInterface()

Block 7: Initialize and Display Interface

In [39]:
# Cell 7: Initialize and Display Interface
# Display the main interface
interface.display()

# Show system status
print("\n" + "="*60)
print("🚀 Multilingual Learning Assistant Ready!")
print("="*60)
print(f"📚 Total words learned: {len(storage_service.get_all_words())}")
print(f"🌍 Supported languages: {len(Config.LANGUAGE_MAPPINGS)}")
print(f"�� Translation models: {'✅' if translation_service.translator else '❌'}")
print(f"🧠 Gemma model: {'✅' if gemma_service.model else '❌'}")
print(f"💾 Storage: {'✅' if storage_service.index else '❌'}")
print("="*60)


🚀 Multilingual Learning Assistant Ready!
📚 Total words learned: 14
🌍 Supported languages: 16
�� Translation models: ✅
🧠 Gemma model: ✅
💾 Storage: ✅


Block 8: Utility Functions

In [40]:
# Cell 8: Utility Functions
def export_learned_words(filename: str = None):
    """Export all learned words to CSV"""
    if filename is None:
        filename = f"learned_words_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    words = storage_service.get_all_words()
    if words:
        df = pd.DataFrame(words)
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"✅ Exported {len(words)} words to {filename}")
    else:
        print("❌ No words to export")

def get_language_statistics():
    """Get statistics by language"""
    words = storage_service.get_all_words()
    if words:
        df = pd.DataFrame(words)
        stats = df['language'].value_counts()
        print("\n📊 Language Statistics:")
        for lang, count in stats.items():
            print(f"  {lang}: {count} words")
    else:
        print("❌ No words found")

def clear_all_data():
    """Clear all stored data (use with caution!)"""
    confirm = input("Are you sure you want to delete all learned words? (yes/no): ")
    if confirm.lower() == 'yes':
        try:
            os.remove(Config.FAISS_INDEX_PATH)
            os.remove(Config.METADATA_PATH)
            storage_service.metadata = []
            storage_service.index = None
            print("✅ All data cleared")
        except Exception as e:
            print(f"❌ Error clearing data: {e}")
    else:
        print("❌ Operation cancelled")


Block 9: Testing

In [41]:
# Cell: Comprehensive Feature Testing
def test_all_features():
    """Test all features of the Multilingual Learning Assistant"""
    
    print("="*60)
    print("COMPREHENSIVE FEATURE TESTING")
    print("="*60)
    
    test_results = {
        'translation': False,
        'gemma_explanation': False,
        'storage': False,
        'search': False,
        'quiz': False,
        'interface': False
    }
    
    # Test 1: Translation Service
    print("\n1. TESTING TRANSLATION SERVICE")
    print("-" * 30)
    try:
        test_word = "नमस्ते"  # Hello in Hindi
        result = translation_service.translate_to_english(test_word, "hindi")
        
        if result['error'] is None and result['translated_text'] != 'unknown':
            print("PASS: Translation service working")
            print(f"   Input: {result['original_text']}")
            print(f"   Output: {result['translated_text']}")
            test_results['translation'] = True
        else:
            print("FAIL: Translation service error")
            print(f"   Error: {result['error']}")
    except Exception as e:
        print(f"FAIL: Translation service exception - {e}")
    
    # Test 2: Gemma Explanation Service
    print("\n2. TESTING GEMMA EXPLANATION SERVICE")
    print("-" * 30)
    try:
        explanation = gemma_service.get_word_explanation("नमस्ते", "hindi", "Hello")
        
        if explanation['error'] is None:
            print("PASS: Gemma explanation service working")
            print(f"   Pronunciation: {explanation['pronunciation']}")
            print(f"   Examples: {len(explanation['usage_examples'])} found")
            test_results['gemma_explanation'] = True
        else:
            print("FAIL: Gemma explanation service error")
            print(f"   Error: {explanation['error']}")
    except Exception as e:
        print(f"FAIL: Gemma explanation service exception - {e}")
    
    # Test 3: Storage Service
    print("\n3. TESTING STORAGE SERVICE")
    print("-" * 30)
    try:
        # Test adding a word
        test_word_data = {
            'original_word': 'नमस्ते',
            'english_translation': 'Hello',
            'language': 'hindi',
            'pronunciation': 'namaste',
            'usage_examples': ['नमस्ते, कैसे हो?'],
            'cultural_context': 'Traditional greeting',
            'related_words': ['प्रणाम', 'स्वागत'],
            'translation_confidence': 0.8,
            'model_used': 'test'
        }
        
        success = storage_service.add_word(test_word_data)
        if success:
            print("PASS: Storage service working")
            print(f"   Words stored: {len(storage_service.get_all_words())}")
            test_results['storage'] = True
        else:
            print("FAIL: Storage service failed to add word")
    except Exception as e:
        print(f"FAIL: Storage service exception - {e}")
    
    # Test 4: Search Service
    print("\n4. TESTING SEARCH SERVICE")
    print("-" * 30)
    try:
        search_results = storage_service.search_words("hello", 5)
        
        if isinstance(search_results, list):
            print("PASS: Search service working")
            print(f"   Search results: {len(search_results)} found")
            test_results['search'] = True
        else:
            print("FAIL: Search service returned invalid results")
    except Exception as e:
        print(f"FAIL: Search service exception - {e}")
    
    # Test 5: Quiz Service
    print("\n5. TESTING QUIZ SERVICE")
    print("-" * 30)
    try:
        quiz_words = storage_service.get_random_words_for_quiz(3)
        
        if isinstance(quiz_words, list):
            print("PASS: Quiz service working")
            print(f"   Quiz words available: {len(quiz_words)}")
            test_results['quiz'] = True
        else:
            print("FAIL: Quiz service returned invalid results")
    except Exception as e:
        print(f"FAIL: Quiz service exception - {e}")
    
    # Test 6: Interface Components
    print("\n6. TESTING INTERFACE COMPONENTS")
    print("-" * 30)
    try:
        # Test if interface components can be created
        from ipywidgets import Dropdown, Text, Button
        
        test_dropdown = Dropdown(options=[('Hindi', 'hindi')])
        test_input = Text(placeholder='Test input')
        test_button = Button(description='Test')
        
        print("PASS: Interface components working")
        print("   Dropdown: Created successfully")
        print("   Text input: Created successfully")
        print("   Button: Created successfully")
        test_results['interface'] = True
    except Exception as e:
        print(f"FAIL: Interface components exception - {e}")
    
    # Test 7: Language Support
    print("\n7. TESTING LANGUAGE SUPPORT")
    print("-" * 30)
    try:
        supported_languages = list(Config.LANGUAGE_MAPPINGS.keys())
        print(f"PASS: Language support working")
        print(f"   Supported languages: {len(supported_languages)}")
        print(f"   Sample languages: {supported_languages[:5]}")
    except Exception as e:
        print(f"FAIL: Language support exception - {e}")
    
    # Test 8: Configuration
    print("\n8. TESTING CONFIGURATION")
    print("-" * 30)
    try:
        print("PASS: Configuration working")
        print(f"   Hugging Face token: {'Set' if Config.HUGGINGFACE_TOKEN else 'Not set'}")
        print(f"   Google AI key: {'Set' if Config.GOOGLE_AI_KEY else 'Not set'}")
        print(f"   Translation model: {Config.DEFAULT_TRANSLATION_MODEL}")
    except Exception as e:
        print(f"FAIL: Configuration exception - {e}")
    
    # Summary
    print("\n" + "="*60)
    print("TESTING SUMMARY")
    print("="*60)
    
    passed_tests = sum(test_results.values())
    total_tests = len(test_results)
    
    for feature, result in test_results.items():
        status = "PASS" if result else "FAIL"
        print(f"{feature.upper()}: {status}")
    
    print(f"\nOVERALL: {passed_tests}/{total_tests} features working")
    
    if passed_tests == total_tests:
        print("STATUS: All features working correctly!")
    elif passed_tests >= total_tests * 0.7:
        print("STATUS: Most features working, minor issues detected")
    else:
        print("STATUS: Multiple features need attention")
    
    return test_results

# Run the comprehensive test
test_results = test_all_features()

COMPREHENSIVE FEATURE TESTING

1. TESTING TRANSLATION SERVICE
------------------------------
PASS: Translation service working
   Input: नमस्ते
   Output: Hello.

2. TESTING GEMMA EXPLANATION SERVICE
------------------------------
PASS: Gemma explanation service working
   Pronunciation: unknown
   Examples: 0 found

3. TESTING STORAGE SERVICE
------------------------------
✅ Added word: नमस्ते (hindi)
PASS: Storage service working
   Words stored: 15

4. TESTING SEARCH SERVICE
------------------------------
PASS: Search service working
   Search results: 5 found

5. TESTING QUIZ SERVICE
------------------------------
PASS: Quiz service working
   Quiz words available: 3

6. TESTING INTERFACE COMPONENTS
------------------------------
PASS: Interface components working
   Dropdown: Created successfully
   Text input: Created successfully
   Button: Created successfully

7. TESTING LANGUAGE SUPPORT
------------------------------
PASS: Language support working
   Supported languages: 16
 